In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
anos = ['2019', '2020', '2021', '2022', '2023']

In [4]:
# Colunas que queremos filtrar nos arquivos CSV
colunas_desejadas = [
    'Data', 
    'Hora UTC', 
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)'
]

def processar_csv(diretorio, arquivo, diretorio_novo):
    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(diretorio, arquivo)
    # Lê o arquivo CSV
    df = pd.read_csv(caminho_arquivo)

    # Remove espaços em branco dos nomes das colunas
    df.columns = df.columns.str.strip()

    # Filtra apenas as colunas que existem no arquivo
    colunas_presentes = [coluna for coluna in colunas_desejadas if coluna in df.columns]
    df_filtrado = df[colunas_presentes]

    # Caminho completo para salvar o novo arquivo CSV
    novo_caminho = os.path.join(diretorio_novo, arquivo)
    df_filtrado.to_csv(novo_caminho, index=False)

    print(f'Arquivo {arquivo} processado. Novo arquivo salvo como {novo_caminho}.')

# Iterar sobre cada ano
for ano in anos:
    diretorio = ano
    diretorio_novo = f'{ano}-formatado'
    
    # Cria o diretório novo se não existir
    os.makedirs(diretorio_novo, exist_ok=True)

    # Lista todos os arquivos CSV no diretório atual
    arquivos_csv = [f for f in os.listdir(diretorio) if f.endswith('.csv')]

    # Processa cada arquivo no diretório
    for arquivo in arquivos_csv:
        processar_csv(diretorio, arquivo, diretorio_novo)

Arquivo 6-2019-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2019-formatado\6-2019-CAMPOSDOSGOYTACAZES.csv.
Arquivo 7-2019-ANGRADOSREIS.csv processado. Novo arquivo salvo como 2019-formatado\7-2019-ANGRADOSREIS.csv.
Arquivo 8-2019-RIODEJANEIRO.csv processado. Novo arquivo salvo como 2019-formatado\8-2019-RIODEJANEIRO.csv.
Arquivo 9-2019-VALENCA.csv processado. Novo arquivo salvo como 2019-formatado\9-2019-VALENCA.csv.
Arquivo 6-2020-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2020-formatado\6-2020-CAMPOSDOSGOYTACAZES.csv.
Arquivo 7-2020-ANGRADOSREIS.csv processado. Novo arquivo salvo como 2020-formatado\7-2020-ANGRADOSREIS.csv.
Arquivo 8-2020-RIODEJANEIRO.csv processado. Novo arquivo salvo como 2020-formatado\8-2020-RIODEJANEIRO.csv.
Arquivo 9-2020-VALENCA.csv processado. Novo arquivo salvo como 2020-formatado\9-2020-VALENCA.csv.
Arquivo 6-2021-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2021-formatado\6-2021-CAMPOSDOSGOYTACAZES.csv.
Arquiv

In [6]:
import pandas as pd
import os

# Lista de anos com suas respectivas pastas
anos = ['2019-formatado', '2020-formatado', '2021-formatado', '2022-formatado', '2023-formatado']

# Função para processar um arquivo CSV
def processar_csv(arquivo_csv, ano):
    # Leitura do CSV
    df = pd.read_csv(arquivo_csv, sep=',')  # ajuste o separador conforme necessário
    
    # Converter a coluna de Data para datetime adequado
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    
    # Converter as colunas numéricas para garantir que possuam valores válidos
    colunas_para_converter = [
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)'
    ]
    
    for coluna in colunas_para_converter:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')  # Converte para numérico, ignorando erros
    
    # Agrupar por semana e calcular a média dos campos relevantes
    df['Semana'] = df['Data'].dt.isocalendar().week
    medias_semanal = df.groupby('Semana').agg({
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'mean',
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'mean',
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'mean',
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'mean'
    }).reset_index()
    
    # Criar o novo caminho para salvar o arquivo
    nova_pasta = os.path.join(ano, 'medias_semanal')
    os.makedirs(nova_pasta, exist_ok=True)  # Cria a pasta, se não existir
    
    # Nome do novo arquivo CSV
    nome_arquivo_novo = os.path.join(nova_pasta, f'medias_semanal_{os.path.basename(arquivo_csv)}')
    
    # Salvar o CSV com as médias semanais
    medias_semanal.to_csv(nome_arquivo_novo, index=False)
    print(f'Salvo: {nome_arquivo_novo}')

# Processar cada arquivo de cada ano
for ano in anos:
    for arquivo in os.listdir(ano):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(ano, arquivo)
            processar_csv(caminho_arquivo, ano)


Salvo: 2019-formatado\medias_semanal\medias_semanal_6-2019-CAMPOSDOSGOYTACAZES.csv
Salvo: 2019-formatado\medias_semanal\medias_semanal_7-2019-ANGRADOSREIS.csv
Salvo: 2019-formatado\medias_semanal\medias_semanal_8-2019-RIODEJANEIRO.csv
Salvo: 2019-formatado\medias_semanal\medias_semanal_9-2019-VALENCA.csv
Salvo: 2020-formatado\medias_semanal\medias_semanal_6-2020-CAMPOSDOSGOYTACAZES.csv
Salvo: 2020-formatado\medias_semanal\medias_semanal_7-2020-ANGRADOSREIS.csv
Salvo: 2020-formatado\medias_semanal\medias_semanal_8-2020-RIODEJANEIRO.csv
Salvo: 2020-formatado\medias_semanal\medias_semanal_9-2020-VALENCA.csv
Salvo: 2021-formatado\medias_semanal\medias_semanal_6-2021-CAMPOSDOSGOYTACAZES.csv
Salvo: 2021-formatado\medias_semanal\medias_semanal_7-2021-ANGRADOSREIS.csv
Salvo: 2021-formatado\medias_semanal\medias_semanal_8-2021-RIODEJANEIRO.csv
Salvo: 2021-formatado\medias_semanal\medias_semanal_9-2021-VALENCA.csv
Salvo: 2022-formatado\medias_semanal\medias_semanal_6-2022-CAMPOSDOSGOYTACAZES.csv